In [ ]:
from openai import OpenAI

client = OpenAI(api_key = "sk-proj")

In [ ]:
# Create a vector store called "Financial Statements"
vector_store = client.vector_stores.create(name="Financial Statements")

# Ready the files for upload to OpenAI
file_paths = ["goog-10k.pdf", "brka-10k.pdf"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

# print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

def get_response(query):
    response = client.responses.create(
        model="gpt-4o",  
        input=query,  
        instructions="You are a financial analyst assistant. Provide insights based on financial statements and online sources.",
        tools=[  # Tools for file search and web search
            {
                "type": "file_search",  # File search tool
                "vector_store_ids": [vector_store.id],  
            },
            {
                "type": "web_search_preview",  # Web search tool
                "user_location": {
                    "type": "approximate",  # Approximate location for context
                    "city": "New York",  # Example city 
                },
            },
        ],
        stream=True,  # Enable streaming for real-time responses
    )
    
    full_response = ""
    for event in response:
        if hasattr(event, "text"):  # Check if the event contains text
            full_response += event.text  # Extract the text field directly
        
    return full_response

# Example of calling the function
query = "What is the latest analysis on Apple's quarterly earnings?"
response = get_response(query)
print(response)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)
Apple's fiscal Q1 2025 earnings report presents a nuanced picture of the company's financial health. The company achieved a record revenue of $124.3 billion, marking a 4% increase from the same quarter in the previous year. Earnings per share (EPS) stood at $2.40, surpassing analyst expectations of $2.35. ([9to5mac.com](https://9to5mac.com/2025/01/30/apple-reports-record-q1-2025-earnings-with-124-30-billion-in-revenue/?utm_source=openai))

**Product Segment Performance:**

- **iPhone:** Sales reached $69.1 billion, slightly below analyst projections of $70.7 billion. This shortfall is attributed to intensified competition and a slower rollout of AI features in certain markets. ([reuters.com](https://www.reuters.com/technology/apple-profit-tops-wall-street-target-even-iphone-china-sales-fall-short-2025-01-30/?utm_source=openai))

- **Mac and iPad:** Both segments exhibited robust growth, with Mac revenue in